In [ ]:
import requests
import re
from bs4 import BeautifulSoup

row_num = 48
book_links = []
for x in range(1, 8):
    url = ""
    if x == 1:
        url = "https://calisphere.org/collections/26771/?q=&sort=a&rows=48"
    elif x >= 2:
        url = "https://calisphere.org/collections/26771/?q=&sort=a&rows=48&start=" + str(row_num)
        row_num += 48

    r = requests.get(url)
    soup = BeautifulSoup(r.content)

    for a in soup.find_all('a', class_='thumbnail__link js-item-link'):
        book_ref =  "https://calisphere.org" + a['href']
        book_links.append(book_ref)
        
    
       
  

In [ ]:
book_metadata = {}

for link in book_links:
    r = requests.get(link)
    soup = BeautifulSoup(r.content)

    book_key = link.split("/")[6]

    md_key = []
    md_val = []
    md_dict = {}

    for dt in soup.find_all('dt', class_='meta-block__type'):  
        md_key.append(dt.get_text())
    
    for dd in soup.find_all('dd', class_='meta-block__defin'):  
        md_val.append(dd.get_text().strip().replace("\n", ""))

    for i, key in enumerate(md_key):
        md_dict[md_key[i]] = md_val[i]

    book_metadata[book_key] = md_dict
    print(link)


In [ ]:
page_links = []
for url in book_links:
    r = requests.get(url)
    soup = BeautifulSoup(r.content)

    page_count_txt = soup.find('div', class_='carousel-complex__navigation-text').text.strip()
    
    page_count = re.findall(r'\d+', page_count_txt)[0]
    if int(page_count) <= 450:
        for p in range(int(page_count)):
            print(url + '?order=' + str(p))
            page_links.append(url + '?order=' + str(p))      


In [ ]:
img_links = {}
for page in page_links:
    r = requests.get(page)
    soup = BeautifulSoup(r.content)


    find_img_link = soup.find(id="js-downloadBtn")

    img_link = ''
    if find_img_link is not None:
        img_link = find_img_link['href']
        img_links[page] = img_link
    else:
        img_links[page] = "NA"

    print(page)
    print(img_link)
    

In [ ]:
import requests
# from time import sleep

for key in img_links.keys():
    if img_links[key] != "NA":
        split_url = key.split("/")
        pg_num = re.findall(r'\d+', split_url[7])[0]
        ref_num = split_url[6]

        filename = "/home/cbrou/Desktop/calisphere_imgs/" + pg_num + "-" + ref_num + ".jpg"

        get_img = requests.get(img_links[key])
        if get_img.status_code == 200:
            f = open(filename,'wb')
            f.write(get_img.content)
            f.close()

        print(filename)

In [ ]:
import os
import json

from exif import Image

for key in book_metadata.keys():
    directory = '/home/cbrou/Desktop/calisphere_imgs'
    print(key)
    
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)

        if key in filename:
            with open(f, 'rb') as image_file:
                    img = Image(image_file)

            img.user_comment = json.dumps(book_metadata[key])

            with open(f, 'wb') as new_image_file:
                    new_image_file.write(img.get_file())    
